<a href="https://colab.research.google.com/github/zombe962/Pythoin-Math-For-AI-100/blob/master/12.%20%EC%88%9C%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 12.1. MNIST 데이터 입력

import numpy as np
from keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()
t_trainlbl, t_testlbl = t_train, t_test

# 28x28을 784로 수정
x_train = x_train.reshape(60000, 784)   #주석 (1) 60000x28x28 크기의 행렬을 60000x784로 고침.
x_test = x_test.reshape(10000, 784)

# one-hot label # 주석 (2)
T0 = np.zeros((t_train.size, 10))   #(60000,10) = 000
T1 = np.zeros((t_test.size, 10))    #(10000,10) = 000

for idx in range(t_train.size): T0[idx][t_train[idx]] = 1   # 주석 (3)
for idx in range(t_test.size): T1[idx][t_test[idx]] = 1

t_train, t_test = T0, T1 

# normalize 0.0 ~ 1.0
x_train = x_train / 255
x_test = x_test / 255

print('MNIST DataSets 준비 완료')

11493376/11490434 [==============================] - 0s 0us/step
MNIST DataSets 준비 완료


\#(1)은 60000x28x28 크기의 행렬을 60000x784로 고치는 부분입니다.

\#(2)는 t_train_size만큼의 크기를 가진 0으로 채운 1차원 행렬(벡터)을 만드는 것

\#(3)은 아래의 for문을 한 줄로 표현한 것입니다.

In [5]:
# 12.2. 함수 정의

# 미분함수
def numerical_diff(f, x):
  h = 1e-4  #0.0001
  nd_coef = np.zeros_like(x)
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:
    index = it.multi_index
    tmp = x[index]
    x[index] = tmp + h
    fxh2 = f()  # f(x+h)
    x[index] = tmp - h
    fxh1 = f()  # f(x-h)
    nd_coef[index] = (fxh2 - fxh1) / (2*h)
    x[index] = tmp
    it.iternext()
  return nd_coef

# 시그모이드
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# 소프트맥스
def softmax(x):
  if x.ndim == 1:   # 기본 1개 처리과정, 백터 입력
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))
  if x.ndim == 2:   # 배치용 n개 처리, 행렬 입력
    x = x.T - np.max(x.T, axis = 0)
    return (np.exp(x) / np.sum(np.exp(x), axis=0)).T

# 크로스엔트로피오차
def cee(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)  # 크기가 1xN인 2차원 행렬로 재구성
    y = y.reshape(1, y.size)
  result = -np.sum(t*np.log(y + 1e-7)) / y.shape[0]
  return result

In [14]:
#12.3. 프로세스별 클래스 생성

class Relu:
  def __init__(self):
    self.mask = None
  
  def forward(self, x):
    self.mask = (x <= 0)
    result = x.copy()
    result[self.mask] = 0
    return result

class Sigmoid:
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    self.out = sigmoid(x)
    return self.out

class Affine:   # wx+b를 계산하여 결과 반환
  def __init__(self, W, b):
    self.W = W  # W0, W1
    self.b = b  # b0, b1
    self.x = None
    self.dW = None  # w0, w1의 기울기
    self.db = None  # b0, b1의 기울기

  def forward(self, x):
    self.x = x
    result = np.dot(self.x, self.W) + self.b
    return result

class SoftmaxWithLoss:
  def __init__(self):
    self.y = None   # 출력(계산결과)
    self.t = None   # 결과(MNIST레이블)
  
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    result = cee(self.y, self.t)
    return result

In [17]:
# 12.4. 네트워크 클래스 생성

class SimpleNetwork:
  def __init__(self, inputx, hidden, outy, weight):
    # 가중치 초기화
    self.netMat = {}
    self.netMat['W0'] = weight * np.random.randn(inputx, hidden)
    self.netMat['b0'] = np.zeros(hidden)
    self.netMat['W1'] = weight * np.random.randn(hidden, outy)
    self.netMat['b1'] = np.zeros(outy)

    # 계층 생성
    self.netLayers = {}
    self.netLayers['Affine1'] = Affine(self.netMat['W0'],
                                       self.netMat['b0'])
    self.netLayers['Relu1'] = Relu()
    self.netLayers['Affine2'] = Affine(self.netMat['W1'],
                                       self.netMat['b1'])
    self.netLayers['Softmax'] = SoftmaxWithLoss()

  def predict(self, x):
    x = self.netLayers['Affine1'].forward(x)
    x = self.netLayers['Relu1'].forward(x)
    x = self.netLayers['Affine2'].forward(x)
    return x

  # x: 입력데이터, t: 정답레이블
  def loss(self, x, t):
    y = self.predict(x)
    return self.netLayers['Softmax'].forward(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t= np.argmax(t, axis=1)
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self, x, t):
    lossfunc = lambda : self.loss(x, t)
    grads = {}
    
    grads['W0'] = numerical_diff(lossfunc, self.netMat['W0'])
    grads['b0'] = numerical_diff(lossfunc, self.netMat['b0'])
    grads['W1'] = numerical_diff(lossfunc, self.netMat['W1'])
    grads['b1'] = numerical_diff(lossfunc, self.netMat['b1'])
    return grads

In [21]:
# 12.5. 미분을 이용한 학습과 검증

import time   # Using Time Module(시간 측정)
t1 = time.time()  # save nowTime (현재 시간 측정)

train_size = x_train.shape[0]   # size of TrainData (입력데이터 크기) 60000
lr = 0.1  #learning rate(학습률)
iter = 0  #Iternation Number (반복 횟수)

iters_num = 1000
batch_size = 20
iter_per_epoch = 1

network = SimpleNetwork(inputx=784, hidden=50, outy=10, weight = 0.2)

print('loss = _______ time = _______ n = ______ | [TrainAcc] [TestAcc]')

for i in range(iters_num):   #1000
  batch_mask = np.random.choice(train_size, batch_size)   # (1) 주석
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch) # 수치미분(편미분) 방식

  # 갱신
  for key in ('W0', 'b0', 'W1', 'b1'): network.netMat[key] -= lr*grad[key]

  loss = network.loss(x_batch, t_batch)

  train_acc = network.accuracy(x_train, t_train)
  test_acc = network.accuracy(x_test, t_test)
  iter = iter + 1

  print('loss = {:7.4f}  '.format(loss), end='')
  print('time = {:8.4f}  '.format(time.time()-t1), end='')
  print('n = {:06d} |  {:6.4f}  {:9.4f}'.format(iter, train_acc, test_acc))

loss = _______ time = _______ n = ______ | [TrainAcc] [TestAcc]
loss =  2.3851  time =  24.5611  n = 000001 |  0.1407     0.1472
loss =  1.7128  time =  48.9964  n = 000002 |  0.1708     0.1739
loss =  1.6308  time =  74.5130  n = 000003 |  0.2000     0.2086
loss =  1.5737  time =  99.2929  n = 000004 |  0.2514     0.2668
loss =  1.9560  time = 123.7933  n = 000005 |  0.2662     0.2832


KeyboardInterrupt: ignored

#time()함수 사용법


```
import time    #1. time을 import함
t1 = time.time()   #2. time.time()으로 현재의 시간(UTC)을 지정함

# 프로그램

print(time.time()-t1)   #3. 현재 시간과 과거 시간을 뺀 값을 출력
```





```
batch_mask = np.random.choice(train_size, batch_size)  #(1) 주석
```

주석 #(1) : 주석의 내용은 다음과 같다. train_size는 60,000이고, batch_size는 20이다. 즉, 0에서 59,999 사이의 숫자 중 임의로 20개 선택한다. 그것을 batch_mask에 넣는다.
